<a href="https://colab.research.google.com/github/kushv16/Audio-Sentiment-Analysis/blob/master/EMODB_Feature_Merging_and_CrossCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install noisereduce

import pandas as pd
import numpy as np
import random
from glob import glob
import os
import librosa
import noisereduce as nr

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df = pd.read_csv('/content/drive/MyDrive/LY Project/Dataframes/Dataset/emov-db-df.csv')
print('The dataset has {} audio files'.format(df.shape[0]))
df.head(5)

The dataset has 6893 audio files


,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,0,/kaggle/input/emovdb-sorted/bea/Amused/amused_...,amused_1-15_0001,EmoV_DB,4.385,44100,female,30,amused
1,1,/kaggle/input/emovdb-sorted/bea/Amused/amused_...,amused_1-15_0002,EmoV_DB,4.944,44100,female,30,amused
2,2,/kaggle/input/emovdb-sorted/bea/Amused/amused_...,amused_1-15_0003,EmoV_DB,6.399,44100,female,30,amused
3,3,/kaggle/input/emovdb-sorted/bea/Amused/amused_...,amused_1-15_0004,EmoV_DB,4.058,44100,female,30,amused
4,4,/kaggle/input/emovdb-sorted/bea/Amused/amused_...,amused_1-15_0005,EmoV_DB,3.358,44100,female,30,amused


In [5]:
paths = list(df['path'])

updated_paths = []
updated_filenames = []

for path in paths:
  path_components = path.split('/')
  updated_path_components = path_components[4:]
  path_string = "/"
  a = "/content/drive/MyDrive/Emov-db/" + path_string.join(updated_path_components)
  updated_paths.append(a)
  
  components = a.split('/')
  updated_filename = components[5] + "_" + components[-1]
  updated_filenames.append(updated_filename)
  
df['path'] = updated_paths
df['filename'] = updated_filenames
df.head(5)

,Unnamed: 0,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,0,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0001.wav,EmoV_DB,4.385,44100,female,30,amused
1,1,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0002.wav,EmoV_DB,4.944,44100,female,30,amused
2,2,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0003.wav,EmoV_DB,6.399,44100,female,30,amused
3,3,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0004.wav,EmoV_DB,4.058,44100,female,30,amused
4,4,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0005.wav,EmoV_DB,3.358,44100,female,30,amused


In [26]:
df_fund = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/EMO/emodb_fund.csv", index_col=0)
df_misc = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/EMO/emo_misc_df.csv", index_col=0)
df_mfcc =  pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/EMO/emo_mfcc_df.csv", index_col=0)

In [27]:
def clean_path(df_column):
  updated_list = []
  for entry in df_column:
    updated_filename = entry.split('/')[-1]
    updated_list.append(updated_filename)
  return updated_list

In [28]:
df_mfcc['filename'] = clean_path(df_mfcc['filename'])
df_fund['filename'] = clean_path(df_fund['filename'])
df_misc['filename'] = clean_path(df_misc['filename'])

In [29]:
df_misc.head()

,zcr_mean,zcr_var,spectral_centroid_mean,spectral_centroid_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,rms_mean,rms_var,...,chroma_cqt_var,chroma_cens_mean,chroma_cens_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,harmony_mean,harmony_var,filename
0,0.162007,0.021054,2964.144668,2.339122e+06,19.103713,43.701505,0.052604,0.005626,0.077632,0.005704,...,0.093189,0.264336,0.013460,1806.577874,165567.287934,4062.025207,2.894398e+06,-0.000056,0.002619,sam_disgust_477-504_0490_cleaned.wav
1,0.098680,0.003406,2188.691919,6.788502e+05,18.732004,40.097979,0.025789,0.002115,0.070617,0.006482,...,0.092350,0.268463,0.011261,1700.854867,142886.476017,3340.401786,2.094917e+06,-0.000032,0.002177,sam_disgust_477-504_0491_cleaned.wav
2,0.137342,0.009106,2483.215859,1.038118e+06,18.587457,36.964738,0.044928,0.005065,0.061328,0.004294,...,0.089461,0.275401,0.007488,1647.832498,151790.490654,3396.530512,2.178150e+06,-0.000030,0.000990,sam_disgust_477-504_0492_cleaned.wav
3,0.174834,0.034542,2983.807578,4.133468e+06,18.780242,38.626912,0.048147,0.005488,0.078340,0.004457,...,0.089136,0.271043,0.009869,1693.035057,185557.116053,3791.489110,4.236041e+06,-0.000056,0.003282,sam_disgust_477-504_0494_cleaned.wav
4,0.125039,0.011741,2530.673930,1.825566e+06,18.773217,40.267877,0.049107,0.006600,0.076856,0.003165,...,0.093474,0.270004,0.010431,1651.317973,128859.486582,3410.704496,2.554006e+06,-0.000026,0.002267,sam_disgust_477-504_0496_cleaned.wav


In [30]:
merged_df = pd.merge(df_fund, df_misc,  on="filename")
merged_df = pd.merge(merged_df, df_mfcc, on='filename')


# Drop the duplicate filename columns
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]


# Save the merged dataframe to a CSV file
# merged_df.to_csv('path/to/merged_df.csv', index=False)
print(merged_df.columns)
merged_df.head()

Index(['f0_mean', 'f0_median', 'f0_std', 'f0_0', 'f0_25', 'f0_75', 'f0_100',
       'filename', 'zcr_mean', 'zcr_var', 'spectral_centroid_mean',
       'spectral_centroid_var', 'spectral_contrast_mean',
       'spectral_contrast_var', 'spectral_flatness_mean',
       'spectral_flatness_var', 'rms_mean', 'rms_var', 'chroma_stft_mean',
       'chroma_stft_var', 'chroma_cqt_mean', 'chroma_cqt_var',
       'chroma_cens_mean', 'chroma_cens_var', 'spectral_bandwidth_mean',
       'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'harmony_mean',
       'harmony_var', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean', 'mfcc4_mean',
       'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean', 'mfcc9_mean',
       'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean', 'mfcc13_mean',
       'mfcc14_mean', 'mfcc15_mean', 'mfcc16_mean', 'mfcc17_mean',
       'mfcc18_mean', 'mfcc19_mean', 'mfcc20_mean', 'mfcc21_mean',
       'mfcc22_mean', 'mfcc23_mean', 'mfcc24_mean', 'mfcc25_mean',
       'mfcc26_mean', 'mf

,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,filename,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,155.707956,149.830708,33.219782,104.729412,131.950791,178.953882,239.219224,sam_disgust_477-504_0490_cleaned.wav,0.162007,0.021054,...,44.988987,59.005190,38.546840,45.735730,85.020610,76.932370,67.000590,80.296560,141.667330,146.925190
1,157.765194,157.371325,35.428211,61.557221,136.604026,175.118722,251.984210,sam_disgust_477-504_0491_cleaned.wav,0.098680,0.003406,...,61.937252,56.860120,74.324360,75.635710,87.359140,61.637848,70.910460,65.519350,85.675560,99.723114
2,155.309180,145.146121,29.352529,107.177346,129.683955,179.211928,216.845374,sam_disgust_477-504_0492_cleaned.wav,0.137342,0.009106,...,49.708620,68.081110,41.700054,37.423670,53.697372,52.938286,67.744010,112.566310,50.453842,75.798810
3,159.765691,157.825829,25.070842,100.579294,145.565318,177.153504,218.101547,sam_disgust_477-504_0494_cleaned.wav,0.174834,0.034542,...,38.689056,31.594845,48.966213,48.995724,48.582462,79.516620,59.368626,90.194496,63.212160,85.197235
4,152.067045,148.967746,28.356369,97.153194,128.194408,175.118722,205.860447,sam_disgust_477-504_0496_cleaned.wav,0.125039,0.011741,...,65.729950,41.032270,63.416880,36.139530,58.152300,53.967070,56.646072,70.302560,79.908660,121.162920


In [31]:
col = merged_df.pop('filename')
merged_df.insert(0, 'filename', col)

merged_df.head(5)

,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,sam_disgust_477-504_0490_cleaned.wav,155.707956,149.830708,33.219782,104.729412,131.950791,178.953882,239.219224,0.162007,0.021054,...,44.988987,59.005190,38.546840,45.735730,85.020610,76.932370,67.000590,80.296560,141.667330,146.925190
1,sam_disgust_477-504_0491_cleaned.wav,157.765194,157.371325,35.428211,61.557221,136.604026,175.118722,251.984210,0.098680,0.003406,...,61.937252,56.860120,74.324360,75.635710,87.359140,61.637848,70.910460,65.519350,85.675560,99.723114
2,sam_disgust_477-504_0492_cleaned.wav,155.309180,145.146121,29.352529,107.177346,129.683955,179.211928,216.845374,0.137342,0.009106,...,49.708620,68.081110,41.700054,37.423670,53.697372,52.938286,67.744010,112.566310,50.453842,75.798810
3,sam_disgust_477-504_0494_cleaned.wav,159.765691,157.825829,25.070842,100.579294,145.565318,177.153504,218.101547,0.174834,0.034542,...,38.689056,31.594845,48.966213,48.995724,48.582462,79.516620,59.368626,90.194496,63.212160,85.197235
4,sam_disgust_477-504_0496_cleaned.wav,152.067045,148.967746,28.356369,97.153194,128.194408,175.118722,205.860447,0.125039,0.011741,...,65.729950,41.032270,63.416880,36.139530,58.152300,53.967070,56.646072,70.302560,79.908660,121.162920


In [32]:
merged_df.sample(10)

,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
3041,jenie_neutral_197-364_0326_cleaned.wav,193.026504,192.074589,31.741521,88.066587,184.997531,208.855628,246.228883,0.157796,0.014804,...,45.214104,27.570396,58.263650,58.311745,82.920260,81.501770,128.46211,219.289200,335.554230,314.962600
1193,sam_anger_225_252_0233_cleaned.wav,204.458459,202.323888,25.561878,135.034995,189.048253,224.169164,254.912125,0.139950,0.016594,...,83.992744,100.129420,101.380120,181.232730,179.508670,196.945240,315.10184,464.825200,396.446700,331.350400
4917,jenie_neutral_197-364_0202_cleaned.wav,202.398130,196.564120,20.964128,164.338063,186.876849,215.596437,259.367911,0.168080,0.031690,...,39.964485,45.817024,92.368614,120.623330,130.868740,152.165540,184.59018,290.871250,331.569670,312.885440
2361,josh_sleepiness_169-196_0196_cleaned.wav,181.572396,190.418372,25.533393,131.190810,155.792315,199.141030,244.810709,0.117666,0.014793,...,39.914394,44.996420,47.316185,36.868618,53.902237,46.776054,59.01708,89.342170,81.036860,75.148870
4424,jenie_anger_369-392_0372_cleaned.wav,481.443240,386.374532,251.159455,211.892619,342.238010,518.735822,1233.768660,0.209816,0.014399,...,251.553270,169.291080,193.437380,97.501920,147.064830,126.114450,107.76719,83.207580,78.552320,130.839460
3564,jenie_sleepiness_393-420_0408_cleaned.wav,208.559088,216.845374,32.391129,91.700404,202.909913,225.792881,262.381621,0.126332,0.022282,...,43.538597,70.475230,46.868080,75.378360,91.326546,91.452286,82.68120,115.676994,150.149250,128.685150
2811,sam_amused_309-336_0322_cleaned.wav,152.950335,142.240602,67.174300,107.798218,128.194408,153.332834,672.717132,0.177315,0.041509,...,30.162762,47.878380,39.922110,34.401540,72.540054,27.762022,67.17853,45.509280,40.499374,32.657627
4443,jenie_anger_369-392_0391_cleaned.wav,360.657151,354.307008,63.223430,240.605007,312.026090,384.149178,582.261273,0.196710,0.007255,...,176.819630,207.633270,151.393050,110.330800,113.988570,69.758680,106.64455,87.201840,77.306496,137.845470
4239,jenie_anger_197-224_0209_cleaned.wav,312.443264,311.127521,53.211771,177.153504,270.461114,358.423857,406.991877,0.173376,0.008621,...,340.337040,292.995400,426.592500,248.560030,219.996730,218.792180,168.63116,137.393680,132.683180,157.293640
1988,sam_disgust_477-504_0478_cleaned.wav,133.465154,132.715174,27.616169,67.908622,113.550443,148.967746,209.458825,0.187227,0.026390,...,52.233162,39.866623,39.026573,28.109217,32.529830,30.518564,37.61328,31.749746,32.618270,65.830480


In [33]:
merged_df.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/EMO/concatenated_features_emo_with_filename.csv", index=False)

In [3]:
merged_df = pd.read_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/EMO/concatenated_features_emo_with_filename.csv")

In [4]:
merged_df.sample(3)

,filename,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,zcr_mean,zcr_var,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
3163,jenie_neutral_57-84_0058_cleaned.wav,207.585561,205.860447,31.020761,156.013045,184.463239,232.409117,299.661415,0.149467,0.019466,...,75.29422,53.549774,79.29110,116.55505,120.17335,241.68770,360.843780,447.90274,280.96558,193.43001
1085,sam_anger_113-140_0123_cleaned.wav,191.174147,187.690730,31.354541,124.544962,166.488345,216.533140,250.532888,0.159392,0.016964,...,83.19956,118.302530,132.74179,145.77520,113.81084,109.89450,160.647840,290.65625,208.12694,248.95068
1071,sam_anger_1-28_0025_cleaned.wav,176.692886,175.118722,33.614709,107.177346,151.136419,196.848791,249.089924,0.132179,0.009971,...,76.28350,86.461464,146.74815,159.74217,99.88296,126.22512,118.145744,226.00465,223.16951,151.28722


In [14]:
key = 'mfcc9_mean'
print(merged_df[key][3163], merged_df[key][1085], merged_df[key][1071])

-7.423577 -13.138178 -11.273848


In [11]:
def extract_mfcc(file):
    
    mfcc_mean, mfcc_var = [], []
    
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    
    # get 20 mean values
    mfcc_mean.append(np.mean(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30).T, axis=0))
    
    # get 20 variance values
    mfcc_var.append(np.var(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30).T, axis=0))
    
    return np.hstack((mfcc_mean, mfcc_var))[0]


In [12]:
path1 = "/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/" + merged_df['filename'][3163]
path2 = "/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/" + merged_df['filename'][1085]
path3 = "/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/" + merged_df['filename'][1071]
extracted_mfcc = []

extracted_mfcc.append(extract_mfcc(path1))
extracted_mfcc.append(extract_mfcc(path2))
extracted_mfcc.append(extract_mfcc(path3))

# add new features to a new dataframe
name_mfcc_mean, name_mfcc_var = [], []

for i in range(0, 30):
    name_mfcc_mean.append('mfcc'+str(i+1)+'_mean')   # mfcc1_mean, mfcc2_mean, ...
    name_mfcc_var.append('mfcc'+str(i+1)+'_var')     # mfcc1_var, mfcc2_var, ...

name_mfcc = name_mfcc_mean + name_mfcc_var           # concatenate mfcc mean and variance names

# finally put all mfcc values in a dataframe
df_mfcc = pd.DataFrame(extracted_mfcc, columns = name_mfcc)

df_mfcc.head()      # print one sample

,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,...,mfcc21_var,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var
0,-188.302231,87.864708,-20.209360,14.405826,-4.904851,8.762483,-15.670589,0.589725,-7.423577,-13.884990,...,75.294220,53.549774,79.291100,116.555054,120.173347,241.687698,360.843781,447.902740,280.965576,193.430008
1,-215.252121,56.892662,-3.762715,3.968256,-10.940133,9.113268,-24.976490,-9.905448,-13.138178,-8.275853,...,83.199562,118.302528,132.741791,145.775208,113.810837,109.894501,160.647842,290.656250,208.126938,248.950684
2,-221.136414,70.306015,-12.244576,17.510485,1.702699,8.464212,-23.222527,-20.820717,-11.273848,-0.270234,...,76.283501,86.461464,146.748154,159.742172,99.882957,126.225121,118.145744,226.004654,223.169510,151.287216
